# TensorFlow crash course
### **PART 10**

### Protobufs
#### Writing

In [1]:
import tensorflow as tf
from tensorflow.train import Feature, Features, Example
from tensorflow.train import BytesList, FloatList, Int64List

person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id": Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com",
                                                        b"c@d.com"]))
        }
    )
)

with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
    f.write(person_example.SerializeToString())

#### Loading

In [2]:
feature_description = {  # Maps the names of the features and their shape
                       "name":tf.io.FixedLenFeature([], tf.string, default_value=""),
                       "id":tf.io.FixedLenFeature([], tf.int64, default_value=0),
                       "emails":tf.io.VarLenFeature(tf.string)
}

for serialized_example in tf.data.TFRecordDataset(["my_contacts.tfrecord"]):
    parsed_example = tf.io.parse_single_example(serialized_example,
                                                feature_description)

In [3]:
print(parsed_example["name"], parsed_example["id"], tf.sparse.to_dense(parsed_example["emails"], default_value=b""))

tf.Tensor(b'Alice', shape=(), dtype=string) tf.Tensor(123, shape=(), dtype=int64) tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string)


#### Bonus

In [4]:
# To serialize the records all in once use tf.io.parse_example() instead
dataset = tf.data.TFRecordDataset(["my_contacts.tfrecord"]).batch(10)
for serialized_examples in dataset:
    parsed_examples = tf.io.parse_example(serialized_examples, feature_description)